In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.metrics import accuracy_score, precision_score, recall_score

In [ ]:
data = pd.read_csv('fnew_data.csv')

In [ ]:
data['Label'] = np.where(data['Close'].shift(-1) > data['Close'], 1, 0)

# Feature columns and sequence length
feature_columns = ['10_day_MA', '30_day_volatility', 'RSI']
seq_length = 60

#Filtering and sorting data


input_data = data.sort_values(by=['Date', 'Time'])


In [ ]:
def create_sequences_per_outlier(input_data, feature_columns, target_column, seq_length):
    sequences = []
    targets = []

    unique_ids = input_data['outlier_id'].unique() if 'outlier_id' in input_data.columns else [None]

    for oid in unique_ids:
        if oid:
            input_seq = input_data[input_data['outlier_id'] == oid]
        else:
            input_seq = input_data

        # Normalize features
        scaler = MinMaxScaler()
        input_features = scaler.fit_transform(input_seq[feature_columns])

        # Ensure sequence length is exactly `seq_length` for input features
        for i in range(len(input_features) - seq_length):
            sequence = input_features[i:i + seq_length]
            sequences.append(sequence)
            target = input_seq[target_column].iloc[i + seq_length]
            targets.append(target)

    return np.array(sequences), np.array(targets)

In [ ]:
# Create sequences
X, y = create_sequences_per_outlier(input_data, feature_columns, 'Label', seq_length)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [ ]:
# Define the LSTM model
model = Sequential()
model.add(LSTM(100, return_sequences=True, input_shape=(seq_length, len(feature_columns))))
model.add(Dropout(0.2))
model.add(LSTM(100, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(50, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test))

# Make predictions on the test set
y_pred_prob = model.predict(X_test)
y_pred = (y_pred_prob > 0.5).astype(int)

# Calculate evaluation metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)

# Print the results
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")